# Multiverse

In [2]:
import os
import requests
import threading
import time

from concurrent.futures import ThreadPoolExecutor as ThreadPool
from multiprocessing import (
    Process,
    Pool as ProcessPool,
    Queue
)
from threading import Thread

## Poke into Thread/Process

In [3]:
def print_hello_world(name: str) -> None:
    time.sleep(1)
    
    print(f'Hello {name}!')

In [4]:
%%time

t = Thread(target=print_hello_world, args=('World',))

t.start()
t.join()

Hello World!
CPU times: user 0 ns, sys: 3.73 ms, total: 3.73 ms
Wall time: 1 s


In [5]:
%%time

t = Process(target=print_hello_world, args=('World',))

t.start()
t.join()

Hello World!
CPU times: user 2.26 ms, sys: 8 ms, total: 10.3 ms
Wall time: 1.02 s


## "Stranger Memory Things", or "Gde Starushkiny Ochki?"

### Threads Share Memory

In [6]:
RESULTS = dict()

def do_and_save(i: int) -> None:
    RESULTS[i] = i

In [7]:
t1 = Thread(target=do_and_save, args=(1,))
t2 = Thread(target=do_and_save, args=(2,))

for t in [t1, t2]:
    t.start()

for t in [t1, t2]:
    t.join()

In [8]:
RESULTS

{1: 1, 2: 2}

In [9]:
RESULTS = dict()

p1 = Process(target=do_and_save, args=(1,))
p2 = Process(target=do_and_save, args=(2,))

for p in [p1, p2]:
    p.start()

for p in [p1, p2]:
    p.join()

In [10]:
RESULTS

{}

Где результаты?..

### Processes Are Separated

In [11]:
queue = Queue()

def do_and_put(i: int, q: Queue) -> None:
    q.put(i)

In [12]:
p1 = Process(target=do_and_put, args=(1, queue))
p2 = Process(target=do_and_put, args=(2, queue))

for p in [p1, p2]:
    p.start()

for p in [p1, p2]:
    p.join()

In [13]:
while not queue.empty():
    print(queue.get())

1
2


### Sometimes Threads Also Need a Break from Each Other

In [14]:
LOCAL_STUFF = threading.local()

def do_and_save_locally(i):
    LOCAL_STUFF.i = i
    
    print(f'Saved in thread: {LOCAL_STUFF.i}.')

In [15]:
t1 = Thread(target=do_and_save_locally, args=(1,))
t2 = Thread(target=do_and_save_locally, args=(2,))

for t in [t1, t2]:
    t.start()

for t in [t1, t2]:
    t.join()

Saved in thread: 1.
Saved in thread: 2.


In [16]:
try:
    _ = LOCAL_STUFF.i
except AttributeError:
    print('Nothing saved in the main process.')

Nothing saved in the main process.


## Pool: Lord of the Threads (or Processes)

### IO Bound

In [17]:
URLS = 80 * ['http://cs.mipt.ru/advanced_python']

In [23]:
%%time

with requests.get(URLS[0]) as response:
    _ = response.text

CPU times: user 3.46 ms, sys: 440 µs, total: 3.9 ms
Wall time: 71.2 ms


In [24]:
%%time

for url in URLS:
    with requests.get(url) as response:
        _ = response.text

CPU times: user 540 ms, sys: 23 ms, total: 563 ms
Wall time: 7.88 s


In [25]:
def download_info(site: str) -> None:
    with requests.get(url) as response:
        _ = response.text

In [26]:
%%time

threads = list()

for url in URLS:
    t = Thread(target=download_info, args=(url,))
    threads.append(t)

for t in threads:
    t.start()

for t in threads:
    t.join()

CPU times: user 347 ms, sys: 75.4 ms, total: 422 ms
Wall time: 5.34 s


In [40]:
%%time

with ThreadPool(max_workers=2) as pool:
    pool.map(download_info, URLS)

CPU times: user 538 ms, sys: 16.9 ms, total: 554 ms
Wall time: 12.3 s


In [41]:
%%time

with ThreadPool(max_workers=5) as pool:
    pool.map(download_info, URLS)

CPU times: user 536 ms, sys: 33.5 ms, total: 570 ms
Wall time: 8.41 s


In [42]:
%%time

if os.cpu_count() == 1:
    print('One CPU?.. Come back with a better computer.')
else:
    with ProcessPool(processes=2) as pool:
        pool.map(download_info, URLS)

CPU times: user 8.74 ms, sys: 5.1 ms, total: 13.8 ms
Wall time: 14.9 s


### CPU Bound

In [44]:
def count(number: int) -> int:
    return sum(i * i for i in range(1, number))

In [45]:
%%time

p = Process(target=count, args=(5_000_000,))

p.start()
p.join()

CPU times: user 1.16 ms, sys: 8.28 ms, total: 9.44 ms
Wall time: 346 ms


In [46]:
NUMBERS = 20 * [5_000_000]

In [52]:
%%time

for number in NUMBERS:
    count(number)

CPU times: user 7.76 s, sys: 0 ns, total: 7.76 s
Wall time: 7.76 s


In [53]:
%%time

with ThreadPool(5) as pool:
    result = pool.map(count, NUMBERS)

CPU times: user 6.83 s, sys: 22.1 ms, total: 6.85 s
Wall time: 6.75 s


In [54]:
%%time

with ProcessPool(2) as pool:
    result = pool.map(count, NUMBERS)

CPU times: user 0 ns, sys: 15.7 ms, total: 15.7 ms
Wall time: 5.46 s


In [57]:
with ProcessPool(2) as pool:
    result = pool.map(count, [100, 1000, 10000])

In [58]:
result

[328350, 332833500, 333283335000]